In [ ]:
pip install flask pyngrok flask_ngrok


In [ ]:
pip install  torch llm2vec librosa pandas soundfile torchaudio peft timm pyyaml torchao

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.5/9.5 MB 43.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 49.6 MB/s eta 0:00:00
  Created wheel for llm2vec: filename=llm2vec-0.2.3-py3-none-any.whl size=31035 sha256=739e5e55ef4ddac794b28a1ae2d533e1cf115c3b000281174b5c5eaa48259412
  Stored in directory: /root/.cache/pip/wheels/c5/b8/34/1d0160e73ad1f223fecbfc1970760f3ba97d808f6aa11d60a9
Successfully built llm2vec
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.22.1
    Uninstalling tokenizers-0.22.1:
      Successfully uninstalled tokenizers-0.22.1
  Attempting uninstall: transformers
    Found existing installation: transformers 4.56.2
    Uninstalling transformers-4.56.2:
      Successfully uninstalled transformers-4.56.2


In [ ]:
from llm2vec import LLM2Vec
from transformers import AutoTokenizer, AutoModel, AutoConfig
from peft import PeftModel
from torchao.quantization import quantize_, Int8WeightOnlyConfig
import torch

access_token = REDACTED

model_id = "McGill-NLP/LLM2Vec-Sheared-LLaMA-mntp"

tokenizer = AutoTokenizer.from_pretrained(model_id, padding = True, truncation = True, max_length = 512)
config = AutoConfig.from_pretrained(model_id, trust_remote_code=True)

if torch.cuda.is_available():
    print("Using GPU")
# GPU path: use bf16 for speed
    model = AutoModel.from_pretrained(
    model_id,
    trust_remote_code=True,
    config=config,
    torch_dtype=torch.bfloat16,
    device_map="cuda",
    token=access_token,
)
else:
    print("Using CPU")
# CPU path: use float32 first, then quantize
    model = AutoModel.from_pretrained(
    model_id,
    trust_remote_code=True,
    config=config,
    torch_dtype=torch.float32,   # quantization requires fp32
    device_map="cpu",
    token=access_token,
    )

    try:
        from torchao.quantization import quantize_
        print("[INFO] Applying torchao quantization for CPU...")
        quant_config = Int8WeightOnlyConfig(group_size=None)
        print("[INFO] Applying torchao quantization with Int8WeightOnlyConfig...")
        quantize_(model, quant_config)

    except ImportError:
        print("[WARNING] torchao not installed. Run: pip install torchao")
        print("[WARNING] Falling back to non-quantized CPU model.")

l2v = LLM2Vec(model, tokenizer, pooling_mode="mean", max_length=512)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/871 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/434 [00:00<?, ?B/s]

...

attn_mask_utils.py: 0.00B [00:00, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/McGill-NLP/LLM2Vec-Sheared-LLaMA-mntp:
- attn_mask_utils.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.
A new version of the following files was downloaded from https://huggingface.co/McGill-NLP/LLM2Vec-Sheared-LLaMA-mntp:
- modeling_llama_encoder.py
- attn_mask_utils.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


pytorch_model.bin:   0%|          | 0.00/5.38G [00:00<?, ?B/s]

Some weights of the model checkpoint at princeton-nlp/Sheared-LLaMA-1.3B were not used when initializing LlamaEncoderModel: ['lm_head.weight']
- This IS expected if you are initializing LlamaEncoderModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing LlamaEncoderModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


adapter_model.safetensors:   0%|          | 0.00/30.0M [00:00<?, ?B/s]

In [ ]:
from sklearn.decomposition import IncrementalPCA
from sklearn.preprocessing import StandardScaler
from pathlib import Path

import numpy as np
import pickle
import torch
import os
import joblib

# Initialize PCA and StandardScaler globally for training
_pca_trainer = None

class SimplePCATrainer:
    """
    A simple PCA trainer that uses IncrementalPCA to fit data in batches.
    It saves checkpoints every 5 batches and can save the final model.

    Args:
        None

    Returns:
        None

    Attributes:
        pca: The IncrementalPCA model.
        scaler: StandardScaler for normalizing data.
        fitted: Boolean indicating if the model has been initialized.
        batch_count_pca: Counter for the number of batches processed.

    Methods:
        process_batch(vectors): Processes a batch of vectors, fits the PCA model incrementally.
        save_final(model_path): Saves the final PCA model to the specified path.
    """

    # Initialize the trainer
    def __init__(self):
        self.pca = None
        self.scaler = StandardScaler()
        self.fitted = False
        self.batch_count_pca = 0

    def _determine_optimal_components(self, vectors):
        """
        Determine the optimal number of PCA components to retain 95% variance.

        Args:
            vectors: The input data to analyze.
        Returns:
            n_components: The optimal number of components.
        """
        temp_pca = IncrementalPCA()
        temp_pca.fit(vectors)
        cumsum_var = np.cumsum(temp_pca.explained_variance_ratio_)
        n_comp_95 = np.argmax(cumsum_var >= 0.95) + 1
        return min(n_comp_95, vectors.shape[1] // 2)

    def process_batch(self, vectors):
        """
        Process a batch of vectors, fitting the PCA model incrementally.

        Args:
            vectors: The input data batch to process.
        Returns:
            reduced_vectors: The PCA-transformed data.

        Note: This method saves a checkpoint every 5 batches.
        """
        if not self.fitted:
            # First batch - initialize everything
            n_components = self._determine_optimal_components(vectors)
            self.pca = IncrementalPCA(n_components=n_components, batch_size=1000)
            self.scaler.fit(vectors)
            self.fitted = True
            print(f"Initialized PCA with {n_components} components")

        # Process batch
        vectors_scaled = self.scaler.transform(vectors)
        self.pca.partial_fit(vectors_scaled)
        reduced_vectors = self.pca.transform(vectors_scaled)

        self.batch_count_pca += 1

        # Save checkpoint every 5 batches
        if self.batch_count_pca % 5 == 0:
            os.makedirs("pca_checkpoints", exist_ok=True)
            with open(f"pca_checkpoints/checkpoint_batch_{self.batch_count_pca}.pkl", 'wb') as f:
                pickle.dump({'pca': self.pca, 'scaler': self.scaler}, f)
            print(f"Saved checkpoint at batch {self.batch_count_pca}")

        print(f"Processed batch {self.batch_count_pca}, shape: {vectors.shape} -> {reduced_vectors.shape}")
        return reduced_vectors

    def save_final(self, model_path):
        """
        Save the final PCA model to the specified path.

        Args:
            model_path: The file path to save the PCA model.

        Returns:
            None

        Note: Change the model path as needed in the data_config.yml file.
        """
        os.makedirs(os.path.dirname(model_path), exist_ok=True)
        with open(model_path, 'wb') as f:
            pickle.dump({'pca': self.pca, 'scaler': self.scaler}, f)
        print(f"Final model saved to {model_path}. Total variance explained: {np.sum(self.pca.explained_variance_ratio_):.4f}")

## For Single Input
def load_pca_model(vectors, model_path="models/fusion/pca.pkl"):
    """
    Load a pre-trained PCA model and transform the input vectors.

    Args:
        vectors: The input data to transform.
        model_path: The file path of the pre-trained PCA model.

    Returns:
        output: The PCA-transformed data.

    Note: Change the model path as needed in the data_config.yml file (or set the path file as shown above). Can be used for the main program.
    """
    model_path = Path(model_path)
    pca = joblib.load(model_path)
    return pca.transform(vectors)

def l2vec_single_train(l2v, lyrics):
    """
    Encode a single lyric string using the provided LLM2Vec model.

    Args:
        l2v: The LLM2Vec model for encoding lyrics.
        lyrics: A single lyric string to encode.

    Returns:
        vectors: The vector representation of the lyrics.

    """
    vectors = l2v.encode([lyrics]).detach().cpu().numpy()
    return vectors

# For Batch Processing
def l2vec_train(l2v, lyrics_list):
    """
    Encode a list of lyric strings using the provided LLM2Vec model.

    Args:
        l2v: The LLM2Vec model for encoding lyrics.
        lyrics_list: A list of lyric strings to encode.
    Returns:
        vectors: The encoded vector representations of the lyrics.

    Note: This function only encodes the lyrics and does not apply PCA reduction. The PCA reduction can be applied separately in the train.py module.
    """
    with torch.no_grad():
        vectors = l2v.encode(lyrics_list)  # lyrics_list: list of strings
    return vectors

In [ ]:
import getpass
import os
import threading
import numpy as np
import torch
from flask import Flask, request, jsonify
from pyngrok import ngrok, conf

# Setup ngrok Authentication
print("Enter your authtoken, which can be copied from https://dashboard.ngrok.com/get-started/your-authtoken")
conf.get_default().auth_token = getpass.getpass("Ngrok Authtoken: ")

# Initialize Flask + ngrok
app = Flask(__name__)
port = 5000

# Open an ngrok tunnel to the HTTP server
public_url = ngrok.connect(port).public_url
print(f" * ngrok tunnel \"{public_url}\" -> \"http://127.0.0.1:{port}\"")
app.config["BASE_URL"] = public_url

# Task Functions
def process_single(lyrics):
    return l2vec_single_train(l2v, lyrics)

def process_batch(lyrics_list):
    return l2vec_train(l2v, lyrics_list)

# Flask Routes
@app.route("/")
def home():
    return """
    <h1>✅ L2Vec Flask API is running</h1>
    <p>Use:</p>
    <ul>
      <li>POST /single  → { "lyrics": "string" }</li>
      <li>POST /batch   → { "lyrics_list": ["song1", "song2", ...] }</li>
    </ul>
    """

@app.route("/single", methods=["POST"])
def single():
    try:
        data = request.get_json()
        if "lyrics" not in data:
            return jsonify({"error": "Missing 'lyrics' field"}), 400

        lyrics = data["lyrics"]
        vectors = process_single(lyrics)
        return jsonify({"vectors": vectors.tolist()})
    except Exception as e:
        return jsonify({"error": str(e)}), 500

@app.route("/batch", methods=["POST"])
def batch():
    try:
        data = request.get_json()
        if "lyrics_list" not in data:
            return jsonify({"error": "Missing 'lyrics_list' field"}), 400

        lyrics_list = data["lyrics_list"]
        vectors = process_batch(lyrics_list)
        return jsonify({"vectors": vectors.tolist()})
    except Exception as e:
        return jsonify({"error": str(e)}), 500

# Start Flask in a Background Thread
threading.Thread(target=app.run, kwargs={"use_reloader": False}).start()

print("\nYour Colab L2Vec backend is ready!")
print(f"Send requests to:\n  {public_url}/single  → single lyrics\n  {public_url}/batch   → multiple lyrics")


Enter your authtoken, which can be copied from https://dashboard.ngrok.com/get-started/your-authtoken
Ngrok Authtoken: ··········
 * ngrok tunnel "https://unmelodiously-parallelless-danyel.ngrok-free.dev" -> "http://127.0.0.1:5000"

Your Colab L2Vec backend is ready!
Send requests to:
  https://unmelodiously-parallelless-danyel.ngrok-free.dev/single  → single lyrics
  https://unmelodiously-parallelless-danyel.ngrok-free.dev/batch   → multiple lyrics


In [ ]:
import torch

def flush_gpu_cache():
    """Flushes the GPU cache."""
    if torch.cuda.is_available():
        torch.cuda.empty_cache()
        print("GPU cache flushed.")
    else:
        print("No GPU available.")

# Example usage: Call this function whenever you want to flush the cache
# flush_gpu_cache()